In [1]:
initial_prompt = (
        "You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. "
        "Respond with '1' if it is positive, or '0' if it is negative."
    )

# Output format prompt
output_format_prompt = (
    "You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for negative. "
    "Do NOT include any additional text or explanation."
)

In [2]:
output_schema = {
    'key_to_extract': None,  # Set to None for direct output
    'value_mapping': None,   # Set to None for direct mapping
    'regex_pattern': r'^(0|1)$',  # Match the entire output for binary classification
    'use_json_mode': False
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "sentiments.csv"

------------------------------------------------------------------------------------------

In [13]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [14]:
!python -m src.iterative_prompt_optimization.dashboard_generator

/opt/anaconda3/bin/python: Error while finding module specification for 'src.iterative_prompt_optimization.dashboard_generator' (ModuleNotFoundError: No module named 'src')


In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['Text', 'Sentiment'])
eval_data.columns = ['text', 'label']
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=5, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (10, 2)
                                                text  label
0  Was this based on a comic-book? A video-game? ...      1
1  If you ask me the first one was really better ...      0
2  When I was a kid, I loved "Tiny Toons". I espe...      1
3  I hate guns and have never murdered anyone, bu...      0
4  I do not have much to say than this is a great...      1


In [8]:
# best_prompt, best_metrics = optimize_prompt(
#     initial_prompt,
#     output_format_prompt,
#     eval_data,
#     iterations)

In [9]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 107190
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment.    │
│ Respond with '1' if it is positive, or '0' if it is negative.                                                   │
│                                                                                                                 │
│ You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for │
│ negative. Do NOT include any additional text or explanation.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/10
Prediction 1/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 2/10
Prediction 2/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 3/10
Prediction 3/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 4/10
Prediction 4/10: 1 | Ground Truth: 0 ❌ (FP)
Processing text 5/10
Prediction 5/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 6/10
Prediction 6/10: 1 | Ground Truth: 0 ❌ (FP)
Processing text 7/10
Prediction 7/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 8/10
Prediction 8/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 9/10
Prediction 9/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 10/10
Prediction 10/10: 1 | Ground Truth: 1 ✅ (TP)


Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.7143 │
│ Recall              │ 1.0000 │
│ Accuracy            │ 0.8000 │
│ F1-score            │ 0.8333 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘


Analyzing misclassifications...


...

You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. 
Respond with '1' if it is positive, or '0' if it is negative.

╭────────────────╮
│ Output Format: │
╰────────────────╯

You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for 
negative. Do NOT include any additional text or explanation.

                         Comparison of All Iterations                         
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    0.7143 │ 1.0000 │   0.8000 │   0.8333 │              0.0000 │
│     2     │    0.7143 │ 1.0000 │   0.8000 │   0.8333 │              0.0000 │
│     3     │    0.6250 │ 1.0000 │   0.7000 │   0.7692 │              0.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/runs/prompt_optimization_logs_20240926_104642
